In [1]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
%run Functions.py

In [4]:
train = pd.read_csv('data/train_c.csv')
test = pd.read_csv('data/test_c.csv')

In [6]:
train,test = get_additional_features(train,test)

In [13]:
col = test.columns

## Random Forest

In [19]:
from sklearn.ensemble import RandomForestRegressor
rf1 = RandomForestRegressor(n_estimators=1000,max_features=0.95,max_depth=2,n_jobs=-1,random_state=42)

In [21]:
train = train.sample(frac = 1,random_state=42)
rf1_train,rf1_test,ID = get_sklearn_stack_data(rf1,train,col,train['y'],test)

Training 1 Fold
R2 Scored of Fold 1 is 0.594718792522
RMSE of Fold 1 is 7.94241748355
Training 2 Fold
R2 Scored of Fold 2 is 0.436209390353
RMSE of Fold 2 is 10.301804413
Training 3 Fold
R2 Scored of Fold 3 is 0.597217297038
RMSE of Fold 3 is 7.8938036502
Training 4 Fold
R2 Scored of Fold 4 is 0.610218106797
RMSE of Fold 4 is 7.60267070007
Training 5 Fold
R2 Scored of Fold 5 is 0.597466264537
RMSE of Fold 5 is 7.90263448083
Start Training
Calculating In-Bag R2 Score
0.564618508279
Calculating Out-Bag R2 Score
0.567165970249
Calculating In-Bag RMSE
8.36529856463
Calculating Out-Bag RMSE
8.32866614553


In [35]:
save_results(rf1_train,rf1_test,'RandomForest_depth2_0.5646_0.5672.csv')

# Gradient Boosting

In [22]:
from sklearn.ensemble import GradientBoostingRegressor

In [24]:
gb1 = GradientBoostingRegressor(n_estimators=1000,max_features=0.95,learning_rate=0.005,max_depth=4)

In [25]:
gb1_train,gb1_test,ID = get_sklearn_stack_data(gb1,train,col,train['y'],test)

Training 1 Fold
R2 Scored of Fold 1 is 0.568203718172
RMSE of Fold 1 is 8.19811351502
Training 2 Fold
R2 Scored of Fold 2 is 0.437719255244
RMSE of Fold 2 is 10.2880007416
Training 3 Fold
R2 Scored of Fold 3 is 0.603689388526
RMSE of Fold 3 is 7.83012624234
Training 4 Fold
R2 Scored of Fold 4 is 0.569305379461
RMSE of Fold 4 is 7.9917165755
Training 5 Fold
R2 Scored of Fold 5 is 0.573245081584
RMSE of Fold 5 is 8.13691952644
Start Training
Calculating In-Bag R2 Score
0.691918117216
Calculating Out-Bag R2 Score
0.550432564597
Calculating In-Bag RMSE
7.03687026532
Calculating Out-Bag RMSE
8.48897532018


In [26]:
save_results(gb1_train,gb1_test,'GBDT_1000trees_depth4_0.55_0.69.csv')

## Lasso Regression

In [27]:
from sklearn.linear_model import Lasso

In [32]:
las1 = Lasso(alpha=5,random_state=42)

In [33]:
las1_train,las1_test,ID = get_sklearn_stack_data(las1,train,col,train['y'],test)

Training 1 Fold
R2 Scored of Fold 1 is 0.603607303776
RMSE of Fold 1 is 7.8548392245
Training 2 Fold
R2 Scored of Fold 2 is 0.464060815042
RMSE of Fold 2 is 10.0441256478
Training 3 Fold
R2 Scored of Fold 3 is 0.61069213745
RMSE of Fold 3 is 7.76063934172
Training 4 Fold
R2 Scored of Fold 4 is 0.612605885741
RMSE of Fold 4 is 7.57934818996
Training 5 Fold
R2 Scored of Fold 5 is 0.603487405026
RMSE of Fold 5 is 7.843307586
Start Training
Calculating In-Bag R2 Score
0.575435112946
Calculating Out-Bag R2 Score
0.578890709407
Calculating In-Bag RMSE
8.26073140125
Calculating Out-Bag RMSE
8.21645199799


In [34]:
save_results(las1_train,las1_test,'Lasso_Lambda5_0.5754_0.5788.csv')

## xgboost gb:linear

In [38]:
y_mean = np.mean(train.y)

In [46]:
params = {
    'booster': 'gblinear', 
    'eta': 0.005,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1,
    'alpha':20
}

In [47]:
dtrain = xgb.DMatrix(train.drop('y', axis=1), train.y)
dtest = xgb.DMatrix(test)

In [49]:
xgb_cvalid = xgb.cv(params, dtrain, num_boost_round=3000, early_stopping_rounds=20,
    verbose_eval=50, show_stdv=True,seed=42)
xgb_cvalid[['train-rmse-mean', 'test-rmse-mean']].plot()
print('Performance does not improve from '+str(len(xgb_cvalid))+' rounds')

[0]	train-rmse:12.3731+0.307918	test-rmse:12.3693+0.606195
[50]	train-rmse:9.24611+0.401868	test-rmse:9.40644+0.755497
[100]	train-rmse:8.77504+0.416331	test-rmse:9.01093+0.771294
[150]	train-rmse:8.55053+0.422751	test-rmse:8.83131+0.773093
[200]	train-rmse:8.42613+0.426376	test-rmse:8.73782+0.770957
[250]	train-rmse:8.34977+0.428857	test-rmse:8.68414+0.768123
[300]	train-rmse:8.2991+0.430706	test-rmse:8.65085+0.765619
[350]	train-rmse:8.26345+0.432159	test-rmse:8.62886+0.76363
[400]	train-rmse:8.23723+0.433267	test-rmse:8.61348+0.761971
[450]	train-rmse:8.21729+0.434089	test-rmse:8.6022+0.760572
[500]	train-rmse:8.20172+0.434716	test-rmse:8.59362+0.759335
[550]	train-rmse:8.18929+0.43522	test-rmse:8.58694+0.758252
[600]	train-rmse:8.17918+0.435651	test-rmse:8.58171+0.757361
[650]	train-rmse:8.17082+0.436026	test-rmse:8.57754+0.756621
[700]	train-rmse:8.16382+0.436357	test-rmse:8.57418+0.756006
[750]	train-rmse:8.15788+0.43665	test-rmse:8.57143+0.755502
[800]	train-rmse:8.15278+0.43691

In [50]:
xgblinear_stack,xgb_linearpred,ID = get_xgb_stack_data(params,2289,train,col,train.y,test)

Training 1 Fold
R2 Scored of Fold 1 is 0.582538670736
RMSE of Fold 1 is 8.06088274967
Training 2 Fold
R2 Scored of Fold 2 is 0.42429201895
RMSE of Fold 2 is 10.4101144987
Training 3 Fold
R2 Scored of Fold 3 is 0.575668856365
RMSE of Fold 3 is 8.10220761373
Training 4 Fold
R2 Scored of Fold 4 is 0.592077123265
RMSE of Fold 4 is 7.77757809346
Training 5 Fold
R2 Scored of Fold 5 is 0.57902903784
RMSE of Fold 5 is 8.08159017731
Start Training
Calculating In-Bag R2 Score
0.578045021378
Calculating Out-Bag R2 Score
0.550721141431
Calculating In-Bag RMSE
8.23530147777
Calculating Out-Bag RMSE
8.48647462657


In [52]:
save_results(xgblinear_stack,xgb_linearpred,'xgb_gblinear_Rounds2289_0.5507_0.5780.csv')

## Stacking LGBM, Lasso and GBDM then Average with XGB Best

In [133]:
lgbm_train,lgbm_test = read_data('sublgb_depth4_0.56_0.63_All_decomp_Magic_X0_ID.csv')
gbt_train,gbt_test = read_data('subGBDT_1000trees_depth4_0.55_0.69.csv')
las_train,las_test = read_data('subLasso_Lambda5_0.5754_0.5788.csv')

Load xgb train and test

In [134]:
xgb_train,xgb_test = read_data('subxgb_depth4_581Rounds_All_Decomp_magic_features_0.5526_0.6378.csv')

Put four predicted values in a single dataframe.

In [135]:
lgbm_train = pd.DataFrame({'ID':lgbm_train['ID'],'model1':lgbm_train['predicted']})
gbt_train = pd.DataFrame({'ID':gbt_train['ID'],'model2':gbt_train['predicted']})
las_train = pd.DataFrame({'ID':las_train['ID'],'model3':las_train['predicted'],'label':las1_train['label']})
stack_train = lgbm_train.merge(gbt_train,on='ID',how='left')
stack_train = stack_train.merge(las_train,on='ID',how='left')

In [136]:
stack_test = pd.DataFrame({'ID':las_test['ID'],'model1':lgbm_test['y'],'model2':gbt_test['y'],'model3':las_test['y']})

In [77]:
MSE(stack_train['model1'],stack_train['model2'])**0.5

2.1555860694175064

In [78]:
MSE(stack_train['model1'],stack_train['model3'])**0.5

1.7523737578350715

In [79]:
MSE(stack_train['model2'],stack_train['model3'])**0.5

2.3185563971742806

## Do Stacking

In [81]:
col = list(stack_test.columns)[1:]

**xgb as Level2**

In [92]:
params = {
    'eta': 0.005,
    'max_depth': 2,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}

In [93]:
dtrain = xgb.DMatrix(stack_train[col], stack_train['label'])
dtest = xgb.DMatrix(stack_test[col])

In [94]:
xgb_cvalid = xgb.cv(params, dtrain, num_boost_round=2000, early_stopping_rounds=20,
    verbose_eval=50, show_stdv=True,seed=42)
xgb_cvalid[['train-rmse-mean', 'test-rmse-mean']].plot()
print('Performance does not improve from '+str(len(xgb_cvalid))+' rounds')

[0]	train-rmse:12.6393+0.276661	test-rmse:12.6306+0.546747
[50]	train-rmse:11.1513+0.315604	test-rmse:11.1481+0.606903
[100]	train-rmse:10.1362+0.348351	test-rmse:10.1405+0.664943
[150]	train-rmse:9.46144+0.375123	test-rmse:9.47237+0.714573
[200]	train-rmse:9.01967+0.39547	test-rmse:9.03701+0.755494
[250]	train-rmse:8.73225+0.409114	test-rmse:8.75641+0.786875
[300]	train-rmse:8.54457+0.418856	test-rmse:8.58127+0.812517
[350]	train-rmse:8.41962+0.425887	test-rmse:8.46968+0.830191
[400]	train-rmse:8.33491+0.4308	test-rmse:8.40043+0.843365
[450]	train-rmse:8.2771+0.434383	test-rmse:8.35562+0.85248
[500]	train-rmse:8.23688+0.436748	test-rmse:8.32768+0.859046
[550]	train-rmse:8.20816+0.438249	test-rmse:8.31048+0.86311
[600]	train-rmse:8.186+0.439478	test-rmse:8.29975+0.865653
[650]	train-rmse:8.16853+0.44021	test-rmse:8.29276+0.866383
[700]	train-rmse:8.15156+0.439319	test-rmse:8.28927+0.865703
[750]	train-rmse:8.13666+0.437439	test-rmse:8.2873+0.865041
Performance does not improve from 777

In [95]:
Stack_Three_train,Stack_Three_pred,ID = get_xgb_stack_data(params,777,stack_train,col,stack_train['label'],stack_test)

Training 1 Fold
R2 Scored of Fold 1 is 0.578095592816
RMSE of Fold 1 is 8.57702372596
Training 2 Fold
R2 Scored of Fold 2 is 0.484598268616
RMSE of Fold 2 is 9.62303230801
Training 3 Fold
R2 Scored of Fold 3 is 0.582557754053
RMSE of Fold 3 is 7.99946988521
Training 4 Fold
R2 Scored of Fold 4 is 0.654525182047
RMSE of Fold 4 is 7.11136474292
Training 5 Fold
R2 Scored of Fold 5 is 0.565147909149
RMSE of Fold 5 is 8.05278075299
Start Training
Calculating In-Bag R2 Score
0.590568687234
Calculating Out-Bag R2 Score
0.572984941336
Calculating In-Bag RMSE
8.11216919964
Calculating Out-Bag RMSE
8.27273428302


In [96]:
save_results(Stack_Three_train,Stack_Three_pred,'Stacking_gbdt_lgbm_lasso_with_xgb_rounds_777.csv')

## Average Stacking With xgb_best

In [137]:
xgb_train = xgb_train[['ID','predicted']]
xgb_train.columns = ['ID','stacking']
xgb_train.columns = ['ID','model4']

In [102]:
Average_train = Stack_Three_train[['ID','predicted','label']].merge(xgb_train,on='ID',how='left')

In [105]:
Average_test = Stack_Three_pred
Average_test.columns = ['ID','stacking']

In [106]:
Average_test['predicted'] = xgb_test['y']

In [109]:
def weight_data(a,log=False):
    if log:
        Res = np.exp(a*np.log(Average_train['stacking'])+(1-a)*np.log(Average_train['predicted']))
    else:
        Res = a*Average_train['stacking'] + (1-a)*Average_train['predicted']
    print(MSE(Average_train['label'],Res)**0.5)
    print(r2_score(Average_train['label'],Res)**0.5)

In [117]:
weight_data(0.5)

8.36748668293
0.751259417426


In [122]:
weight_data(0.2)
weight_data(0.22)

8.32572114227
0.754140879991
8.32762492163
0.754010088234


In [123]:
a = 0.25
Final = a*Average_test['stacking'] + (1-a)*Average_test['predicted']

In [125]:
sub = pd.DataFrame({'ID':test['ID'],'y':Final})

In [129]:
sub.to_csv('submission/weighted_Stacking_bestxgb_Stacking_0.25.csv',index=False)

## Stack also xgb and rf, then using three different level 2 algorithm before averaging them on level 3

In [138]:
stack_train = stack_train.merge(xgb_train,on='ID',how='left')

In [140]:
rf_train, rf_test = read_data('subRandomForest_depth2_0.5646_0.5672.csv')

In [141]:
rf_train = rf_train[['ID','predicted']]
rf_train.columns = ['ID','model5']

In [142]:
stack_train = stack_train.merge(rf_train,on='ID',how='left')

In [144]:
def Mean_error(list1,list2):
    diff = [x - y for x,y in zip(list1,list2)]
    return np.mean(diff)

In [145]:
for i in range(1,6):
    model = 'model'+str(i)
    label = stack_train['label']
    pred = stack_train[model]
    print('Mean Error')
    print(Mean_error(label,pred))
    print('RMSE')
    print(MSE(label,pred)**0.5)
    print('R2_Score')
    print(r2_score(label,pred))

Mean Error
-0.0206285253464
RMSE
8.41581005274
R2_Score
0.559344778381
Mean Error
-0.0961229167594
RMSE
8.53753861785
R2_Score
0.546505072722
Mean Error
-0.00228340533135
RMSE
8.26780016108
R2_Score
0.574708196375
Mean Error
-0.0401703491356
RMSE
8.49895704984
R2_Score
0.550594544471
Mean Error
-0.00691504290748
RMSE
8.38788325541
R2_Score
0.562264442892


In [146]:
stack_test['model4'] = xgb_test['y']
stack_test['model5'] = rf_test['y']

In [148]:
col = list(stack_test.columns)[1:]

## Meta 1: xgb

In [150]:
params = {
    'eta': 0.005,
    'max_depth': 2,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}

In [151]:
dtrain = xgb.DMatrix(stack_train[col], stack_train['label'])
dtest = xgb.DMatrix(stack_test[col])

In [152]:
xgb_cvalid = xgb.cv(params, dtrain, num_boost_round=2000, early_stopping_rounds=20,
    verbose_eval=50, show_stdv=True,seed=42)
xgb_cvalid[['train-rmse-mean', 'test-rmse-mean']].plot()
print('Performance does not improve from '+str(len(xgb_cvalid))+' rounds')

[0]	train-rmse:12.6393+0.276661	test-rmse:12.6306+0.546747
[50]	train-rmse:11.1513+0.315601	test-rmse:11.1481+0.60695
[100]	train-rmse:10.1361+0.348313	test-rmse:10.1405+0.665117
[150]	train-rmse:9.46125+0.374988	test-rmse:9.47269+0.714465
[200]	train-rmse:9.0192+0.395139	test-rmse:9.03748+0.7551
[250]	train-rmse:8.73144+0.408623	test-rmse:8.75664+0.786292
[300]	train-rmse:8.54331+0.418168	test-rmse:8.58081+0.811773
[350]	train-rmse:8.41782+0.425127	test-rmse:8.4682+0.829371
[400]	train-rmse:8.33267+0.429943	test-rmse:8.39856+0.841971
[450]	train-rmse:8.2745+0.433365	test-rmse:8.3535+0.850414
[500]	train-rmse:8.23386+0.43558	test-rmse:8.32614+0.856924
[550]	train-rmse:8.2039+0.436383	test-rmse:8.3096+0.860843
[600]	train-rmse:8.18117+0.437051	test-rmse:8.29864+0.8637
[650]	train-rmse:8.16292+0.437412	test-rmse:8.29216+0.864659
[700]	train-rmse:8.1412+0.434977	test-rmse:8.28862+0.864294
[750]	train-rmse:8.12225+0.431416	test-rmse:8.28717+0.863467
[800]	train-rmse:8.10755+0.427249	test-r

In [155]:
xgb_level2_train,xgb_level2_test,ID = get_xgb_stack_data(params,803,stack_train,col,stack_train['label'],stack_test)

Training 1 Fold
R2 Scored of Fold 1 is 0.578655565822
RMSE of Fold 1 is 8.5713299048
Training 2 Fold
R2 Scored of Fold 2 is 0.484772004839
RMSE of Fold 2 is 9.62141026243
Training 3 Fold
R2 Scored of Fold 3 is 0.582550223458
RMSE of Fold 3 is 7.9995420395
Training 4 Fold
R2 Scored of Fold 4 is 0.654879519434
RMSE of Fold 4 is 7.10771690996
Training 5 Fold
R2 Scored of Fold 5 is 0.564033077039
RMSE of Fold 5 is 8.06309661923
Start Training
Calculating In-Bag R2 Score
0.58976897611
Calculating Out-Bag R2 Score
0.572978078119
Calculating In-Bag RMSE
8.12008753236
Calculating Out-Bag RMSE
8.27261914718


## Meta 2: Lasso Regression

In [159]:
lasso_level2_train,lasso_level2_test,ID = get_sklearn_stack_data(Lasso(alpha=0.1,random_state=42),stack_train,col,stack_train['label'],stack_test)

Training 1 Fold
R2 Scored of Fold 1 is 0.581512295495
RMSE of Fold 1 is 8.54222353039
Training 2 Fold
R2 Scored of Fold 2 is 0.483842124283
RMSE of Fold 2 is 9.63008868177
Training 3 Fold
R2 Scored of Fold 3 is 0.578875283937
RMSE of Fold 3 is 8.03467610653
Training 4 Fold
R2 Scored of Fold 4 is 0.662492324792
RMSE of Fold 4 is 7.02888734361
Training 5 Fold
R2 Scored of Fold 5 is 0.586485928938
RMSE of Fold 5 is 7.85272230675
Start Training
Calculating In-Bag R2 Score
0.576500406829
Calculating Out-Bag R2 Score
0.578641591489
Calculating In-Bag RMSE
8.25036121518
Calculating Out-Bag RMSE
8.21771959381


## Meta 3: Gradient Boosting

In [165]:
gbdt_level2_train,gbdt_level2_test,ID = get_sklearn_stack_data(GradientBoostingRegressor(n_estimators=100,subsample=0.95,max_features=1,max_depth=2,random_state=42),stack_train,col,stack_train['label'],stack_test)

Training 1 Fold
R2 Scored of Fold 1 is 0.579512163055
RMSE of Fold 1 is 8.56261267447
Training 2 Fold
R2 Scored of Fold 2 is 0.484280098229
RMSE of Fold 2 is 9.62600211908
Training 3 Fold
R2 Scored of Fold 3 is 0.580777719108
RMSE of Fold 3 is 8.01650719839
Training 4 Fold
R2 Scored of Fold 4 is 0.65307364588
RMSE of Fold 4 is 7.12628852712
Training 5 Fold
R2 Scored of Fold 5 is 0.560521796059
RMSE of Fold 5 is 8.09550162391
Start Training
Calculating In-Bag R2 Score
0.600413660334
Calculating Out-Bag R2 Score
0.571633084466
Calculating In-Bag RMSE
8.01404507639
Calculating Out-Bag RMSE
8.28538242859


## Meta 4: LightGBM

In [182]:
%run Functions.py

In [193]:
params = params = {
        'objective': 'regression',
            'metric': 'rmse',
            'boosting': 'gbdt',
            'learning_rate': 0.0045 , #small learn rate, large number of iterations
            'verbose': 0,
            'num_iterations': 500,
            'bagging_fraction': 0.95,
            'bagging_freq': 1,
            'bagging_seed': 42,
            'feature_fraction': 0.95,
            'feature_fraction_seed': 42,
            'max_bin': 100,
            'max_depth': 4,
            'num_rounds': 1200
        }

In [194]:
lgb_train,lgb_test,ID = get_lgb_stack_data(params,1200,stack_train,col,stack_train['label'],stack_test)

Training 1 Fold
R2 Scored of Fold 1 is 0.578675688729
RMSE of Fold 1 is 8.57112522412
Training 2 Fold
R2 Scored of Fold 2 is 0.481436756758
RMSE of Fold 2 is 9.6525013772
Training 3 Fold
R2 Scored of Fold 3 is 0.581304296484
RMSE of Fold 3 is 8.01147092298
Training 4 Fold
R2 Scored of Fold 4 is 0.651904807622
RMSE of Fold 4 is 7.13828311127
Training 5 Fold
R2 Scored of Fold 5 is 0.558957979002
RMSE of Fold 5 is 8.10989214582
Start Training
Calculating In-Bag R2 Score
0.608448568119
Calculating Out-Bag R2 Score
0.570455905719
Calculating In-Bag RMSE
7.93306244146
Calculating Out-Bag RMSE
8.29665455628


## Level 3: LassoCV as Method

In [199]:
stack_level3_train = pd.DataFrame({'ID':xgb_level2_train['ID'],'xgb':xgb_level2_train['predicted'],'label':xgb_level2_train['label']})

In [200]:
stack_level3_train['lgb'] = lgb_train['predicted']
stack_level3_train['gbt'] = gbdt_level2_train['predicted']
stack_level3_train['las'] = lasso_level2_train['predicted']

In [206]:
stack_level3_test = pd.DataFrame({'ID':xgb_level2_test['ID'],'xgb':xgb_level2_test['y']})
stack_level3_test['lgb'] = lgb_test.y
stack_level3_test['gbt'] = gbdt_level2_test['y']
stack_level3_test['las'] = lasso_level2_test['y']

In [208]:
col = ['xgb','lgb','gbt','las']

In [209]:
TL_Stack_TRAIN,TL_Stack_TEST,ID = get_sklearn_stack_data(Lasso(alpha=0.1),stack_level3_train,col,stack_level3_train['label'],stack_level3_test)

Training 1 Fold
R2 Scored of Fold 1 is 0.580873432294
RMSE of Fold 1 is 8.54874132174
Training 2 Fold
R2 Scored of Fold 2 is 0.484079390524
RMSE of Fold 2 is 9.62787505907
Training 3 Fold
R2 Scored of Fold 3 is 0.580037986225
RMSE of Fold 3 is 8.02357678963
Training 4 Fold
R2 Scored of Fold 4 is 0.660281156263
RMSE of Fold 4 is 7.0518744975
Training 5 Fold
R2 Scored of Fold 5 is 0.581570857527
RMSE of Fold 5 is 7.89925358479
Start Training
Calculating In-Bag R2 Score
0.575961601362
Calculating Out-Bag R2 Score
0.577368564567
Calculating In-Bag RMSE
8.25560788694
Calculating Out-Bag RMSE
8.23026425055


In [210]:
save_results(TL_Stack_TRAIN,TL_Stack_TEST,'first_three_level_stacking.csv')

## Final Today: Try to reproduce popular Kernel Results

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import ElasticNetCV, LassoLarsCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, make_union
from sklearn.utils import check_array
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import r2_score



class StackingEstimator(BaseEstimator, TransformerMixin):
    
    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y=None, **fit_params):
        self.estimator.fit(X, y, **fit_params)
        return self
    def transform(self, X):
        X = check_array(X)
        X_transformed = np.copy(X)
        # add class probabilities as a synthetic feature
        if issubclass(self.estimator.__class__, ClassifierMixin) and hasattr(self.estimator, 'predict_proba'):
            X_transformed = np.hstack((self.estimator.predict_proba(X), X))

        # add class prodiction as a synthetic feature
        X_transformed = np.hstack((np.reshape(self.estimator.predict(X), (-1, 1)), X_transformed))

        return X_transformed


train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))



n_comp = 12

# tSVD
tsvd = TruncatedSVD(n_components=n_comp, random_state=420)
tsvd_results_train = tsvd.fit_transform(train.drop(["y"], axis=1))
tsvd_results_test = tsvd.transform(test)

# PCA
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(train.drop(["y"], axis=1))
pca2_results_test = pca.transform(test)

# ICA
ica = FastICA(n_components=n_comp, random_state=420)
ica2_results_train = ica.fit_transform(train.drop(["y"], axis=1))
ica2_results_test = ica.transform(test)

# GRP
grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=420)
grp_results_train = grp.fit_transform(train.drop(["y"], axis=1))
grp_results_test = grp.transform(test)

# SRP
srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=420)
srp_results_train = srp.fit_transform(train.drop(["y"], axis=1))
srp_results_test = srp.transform(test)

#save columns list before adding the decomposition components

usable_columns = list(set(train.columns) - set(['y']))

# Append decomposition components to datasets
for i in range(1, n_comp + 1):
    train['pca_' + str(i)] = pca2_results_train[:, i - 1]
    test['pca_' + str(i)] = pca2_results_test[:, i - 1]

    train['ica_' + str(i)] = ica2_results_train[:, i - 1]
    test['ica_' + str(i)] = ica2_results_test[:, i - 1]

    train['tsvd_' + str(i)] = tsvd_results_train[:, i - 1]
    test['tsvd_' + str(i)] = tsvd_results_test[:, i - 1]

    train['grp_' + str(i)] = grp_results_train[:, i - 1]
    test['grp_' + str(i)] = grp_results_test[:, i - 1]

    train['srp_' + str(i)] = srp_results_train[:, i - 1]
    test['srp_' + str(i)] = srp_results_test[:, i - 1]

#usable_columns = list(set(train.columns) - set(['y']))

y_train = train['y'].values
y_mean = np.mean(y_train)
id_test = test['ID'].values
#finaltrainset and finaltestset are data to be used only the stacked model (does not contain PCA, SVD... arrays) 
finaltrainset = train[usable_columns].values
finaltestset = test[usable_columns].values


'''Train the xgb model then predict the test data'''

xgb_params = {
    'n_trees': 520, 
    'eta': 0.0045,
    'max_depth': 4,
    'subsample': 0.93,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}
# NOTE: Make sure that the class is labeled 'class' in the data file

dtrain = xgb.DMatrix(train.drop('y', axis=1), y_train)
dtest = xgb.DMatrix(test)

num_boost_rounds = 1250
# train model
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)
y_pred = model.predict(dtest)

'''Train the stacked models then predict the test data'''

stacked_pipeline = make_pipeline(
    StackingEstimator(estimator=LassoLarsCV(normalize=True)),
    StackingEstimator(estimator=GradientBoostingRegressor(learning_rate=0.001, loss="huber", max_depth=3, max_features=0.55, min_samples_leaf=18, min_samples_split=14, subsample=0.7)),
    LassoLarsCV()

)


stacked_pipeline.fit(finaltrainset, y_train)
results = stacked_pipeline.predict(finaltestset)

'''R2 Score on the entire Train data when averaging'''

print('R2 score on train data:')
print(r2_score(y_train,stacked_pipeline.predict(finaltrainset)*0.2855 + model.predict(dtrain)*0.7145))

'''Average the preditionon test data  of both models then save it on a csv file'''

sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = y_pred*0.75 + results*0.25
sub.to_csv('submission/Repro_Ker_2.csv', index=False)


/home/jiashen/env3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/jiashen/env3/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=6.025e-02, with an active set of 2 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/home/jiashen/env3/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=5.597e-02, with an active set of 3 regresso

/home/jiashen/env3/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 5 iterations, i.e. alpha=8.590e-03, with an active set of 5 regressors, and the smallest cholesky pivot element being 2.220e-16
  ConvergenceWarning)
/home/jiashen/env3/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 10 iterations, i.e. alpha=5.601e-03, with an active set of 10 regressors, and the smallest cholesky pivot element being 2.220e-16
  ConvergenceWarning)
/home/jiashen/env3/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 11 iterations, i.e. alpha=5.191e-03, with an active set of 11 regressors, and the smallest cholesky pivot element being 2.220e-16
  ConvergenceWarning)
/home/jiashen/env3/lib/pyth

R2 score on train data:
0.658927980335


In [ ]:
R2 score on train data:
0.658887674983

'Average the preditionon test data  of both models then save it on a csv file'